# Tightly Trained Architectures

## 1. Wstęp

W ramach ćwiczenia realizowana jest praca na pięciu architekturach głębokich sieci neuronowych w zadaniach klasyfikacji. W szczególności treść laboratorium poświęcona zostanie analizie modeli przy pracy na danych o charakterze "in the wild", a zatem surowych obrazach zaczerpniętych ze zdjęć różnego rodzaju bydynków z ulic amerykańskich miast oraz zdjęć mieszkań z ogłoszeń internetowych. Głównym celem zadań w tym laboratorium jest badanie w jaki sposób zachowają się wytrenowane sieci (trenowane w ściśle określonej konfiguracji warstw, wielkości oraz hiperaparamterów) na danych surowo pobranych z [Map Google](https://www.google.com/maps) oraz z [Airbnb](https://www.airbnb.pl/). Zbadane zostaną następujące architektury:
1. [VGG-16](https://arxiv.org/pdf/1409.1556v6.pdf) - klasyczna już głęboka sieć konwolucyjna o składająca się z 16 warstw (13 konwolucyjnych i 3 fully-connected), której cechą charakterystyczną  jest zastosowanie wielu warstw konwolucyjnych o niewielkich filtrach (3x3), co przyczynia się do jej zdolności do efektywnego uczenia hierarchicznych cech obrazów.
2. [ResNet50](https://arxiv.org/pdf/2110.00476.pdf) - Residual Network (sieć resztkowa), model głębokiego uczenia, w którym warstwy wagowe uczą się funkcji resztkowych w odniesieniu do danych wejściowych warstwy. Jest specyficznym typem konwolucyjnej sieci neuronowej. Składa sie z 50 warstw (48 konwolucyjnych, jedna MaxPool oraz jedna średniej puli). Posiada mniej filtrów niż VGG oraz jest mniej złożona. Architektura tej sieci opiera się na dwóch zasadach: Liczba filtrów jest taka sama w każdej warstwie w zależności od rozmiaru wyjściowej mapy cech. Jeśli rozmiar mapy cech jest zmniejszony o połowę, ma ona podwójną liczbę filtrów, by utrzymać złożoność czasową każdej warstwy.
3. [SwinTransformer](https://arxiv.org/pdf/2103.14030v2.pdf) - nowy rodzaj transformera wizyjnego, zaprojektowany do ogólnego zastosowania w wizji komputerowej. Charakteryzuje go hierarchiczna architektura, pozwalająca na łączenie sąsiednich fragmentów obrazu w głebszych warstawch. Pozwala ona na skalowanie i wykorzystywania na różnych rozmiarach. W Swin Transformerze, samoorganizacja jest obliczana w obrębie lokalnych, nienakładających się okien. Te okna są przesuwane między kolejnymi warstwami transformatora, co umożliwia połączenia międzyokienne i zwiększa moc modelowania. Dzięki obliczaniu samoorganizacji w ograniczonych lokalnych oknach, osiąga on liniową złożoność obliczeniową względem rozmiaru obrazu, co czyni go skalowalnym. Dzięki tym cechom model osiąga wysoką wydajność w zadaniach tj. klasyfikacja obrazów, detekcja obiektów czy segmentacja.
4. [ConvNeXt](https://openaccess.thecvf.com//content/CVPR2022/papers/Liu_A_ConvNet_for_the_2020s_CVPR_2022_paper.pdf) - konwolucyjna odpowiedź na state-of-the-art sieci transformerowe, gdzie od strony architektonicznej sieć ta jest formą modernizacji sieci ResNet poprzez zastosowanie schematów projektowych sprawdzających się w transfomerach, a więc między innymi: "stem" cell zmieniony na "patchify" cell (nie nakładająca się konwolucja)​, większy kernel size (7x7), inverted bottlneck, depthiwise convolution oraz liczne zmiany micro-designu takie jak nieliniowość GeLU, Layer Normalization, czy Layer Scale.
5. [ConvNeXt V2](https://openaccess.thecvf.com/content/CVPR2023/papers/Woo_ConvNeXt_V2_Co-Designing_and_Scaling_ConvNets_With_Masked_Autoencoders_CVPR_2023_paper.pdf) - modyfikacja oryginalnego ConvNeXt'a a dwa dodatkowe rozwiązania projektowe, mianowicie zastąpienie Layer Scale przez Global Response Normalization (GRN) do promocji różnorodności cech w trakcie treningu, a także co ważniejsze wykorzystanie self-supervised learning'u z wykorzystaniem Masked Autoencoderów (MAE) do pretreningu sieci.

Każda z wspomnianych sieci wytrenowana została z optymalneym zestawieniem hiperparametrów zasugerowanym w oryginalnych artykułach, a proces treningu realizowany był jako transfer learning w którym szkolona była głowa klasyfikacyjna każdej sieci.

Link do checkpointów dla każdego z modeli: https://drive.google.com/drive/folders/1i-3OwNELUbMxv7CGIuQy_xygln_9OwCA

Drugim etapem badań sieci była analiza ich interpretowalności w związku z czym wykorzystana została technika [Integrated Gradients](https://arxiv.org/pdf/1703.01365.pdf).
Jednym z głównych wymagań dotyczących interpretowalności modeli jest aksjomat wrażliwości, który ma dwa główne aspekty:

- Jeśli baseline i dane wejściowe różnią się tylko jedną cechą, ale mają różne przewidywania, wówczas ta cecha otrzymuje niezerowe przypisanie.
- Jeśli cecha nie odgrywa żadnej roli w sieci, nie otrzymuje żadnych przypisań.

Prosta metoda, taka jak mnożenie danych wejściowych przez ich gradient, nie spełnia tego wymogu, na przykład dla $F(x) = 1 - max(0, x)$ wymóg ten nie jest spełniony. Metoda taka jak Integrated Gradients działa dobrze dla tego wymógu. Z definicji są one obliczane w następujący sposób:

$\text{IntegratedGrads}_i(x) = (x - x') \times \int_{\alpha=0}^1 \frac{\partial f(x'+\alpha \times (x-x'))}{\partial x_i} \, d\alpha$

Proces obliczania zintegrowanych gradientów jest następujący:

1. Wybierz linię bazową lub punkt odniesienia dla funkcji wejściowych. Reprezentuje on punkt, w którym dane wyjściowe modelu są znane (zazwyczaj jest to punkt, w którym wszystkie cechy są ustawione na zero - czarny obraz).

2. Utwórz ścieżkę od baseline'u do punktu danych wejściowych. Ścieżka ta jest zwykle linią prostą w przestrzeni wejściowej łączącą baseline z rzeczywistymi danymi wejściowymi - interpolacja między obrazami (baseline i dane wejściowe) za pomocą kroków α od 0 do 1.

3. Oblicz gradienty danych wyjściowych modelu w odniesieniu do cech wejściowych w wielu punktach wzdłuż skonstruowanej ścieżki.

4. Integracja tych gradientów wzdłuż ścieżki. Jest to często wykonywane przy użyciu numerycznych metod całkowania.

5. Zintegrowane gradienty reprezentują wkład każdej cechy w wynik modelu. Wyższe wartości wskazują, że odpowiednia cecha ma bardziej znaczący wpływ na prognozę.

![ComparisonOfModels.png](https://i.postimg.cc/TPMsSt7r/Microsoft-Teams-image.png)
*Rysunek 1. Porównanie dokładności klasyfikacji na zbiorze StreetsAndHouses dla poszczególnych modeli wraz z uwzględnieniem ich wielkości.*

## 2. Konfiguracja środowiska, zmiennych i elementów pomocniczych

In [ ]:
# Do poprawnego działania i szybkiej konfiguracji środowiska sugerujemy pracować na platformie Google Colab

!pip install captum
!pip install datasets
!pip install transformers

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.init as init
import torchvision as tv

from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
from datasets import load_dataset
from tqdm import tqdm
from torchvision import transforms
from torchvision.transforms import v2
from transformers import ConvNextV2ForImageClassification
from transformers.models.convnextv2.modeling_convnextv2 import ConvNextV2Embeddings

sns.set_theme()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [6]:
# Mapowanie predykowanych labeli przez sieci do odpowiednich nazw ze zbioru treningowego
LABELS = {
    0: "apartment",
    1: "bath",
    2: "bed",
    3: "church",
    4: "commercial",
    5: "din",
    6: "garage",
    7: "house",
    8: "industrial",
    9: "kitchen",
    10: "living",
    11: "retail",
    12: "roof" }

In [8]:
# Określenie liczby kanałów w obrazach (3 bo RGB) i liczby predykowanych klas (13)
IN_CHANNELS = 3
N_CLASSES = 13

In [7]:
# Transformacja obrazu do odpowiedniej wielkości, zamiana na tensor i normalizacja (ImageNet-1k defaults)
img_transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## 3. Definicja modeli i wgranie wag

### 3.1. VGG-16

In [ ]:
vgg16_model = tv.models.vgg16(weights="IMAGENET1K_V1")

# Add on classifier
n_inputs = vgg16_model.classifier[6].in_features
vgg16_model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.6),
    nn.Linear(256, N_CLASSES), nn.LogSoftmax(dim=1))

# !!! TODO: Uzupełnij ścieżkę do checkpointa:
vgg16_model.load_state_dict(torch.load(""))

### 3.2. ResNet-50

In [ ]:
resnet50_model = tv.models.resnet50(weights="IMAGENET1K_V1")

# Add on classifier
n_inputs = resnet50_model.fc.in_features
resnet50_model.fc = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.6),
    nn.Linear(256, N_CLASSES), nn.LogSoftmax(dim=1))

# !!! TODO: Uzupełnij ścieżkę do checkpointa:
resnet50_model.load_state_dict("")

### 3.3. Swin Transformer

In [ ]:
swin_model = tv.models.swin_b(weights="IMAGENET1K_V1")

# Add on classifier
n_inputs = swin_model.head.in_features
swin_model.head = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.GELU(), nn.Dropout(0.6),
    nn.Linear(256, N_CLASSES), nn.LogSoftmax(dim=1))

# !!! TODO: Uzupełnij ścieżkę do checkpointa:
resnet50_model.load_state_dict("")

### 3.4. ConvNeXt

In [ ]:
convnext_model = tv.models.convnext_base(weights="IMAGENET1K_V1")

# Add on classifier
n_inputs = convnext_model.classifier[2].in_features
# !!! TODO: Dodaj głowę klasyfikacyjną dla modelu ConvNeXt w analogiczny sposób jak przy poprzednich.
# Wskazówka: Wyprintuj sieć (w osobnej komórce: convnext_model) -> znajdź w którym miejscu znajduje się warstwa
# fully-connected w bloku classifier -> zastąp ją warstwą klasyfikacyjną taka jaka jest zaimplementowana dla ConvNeXt V2.

# !!! TODO: Uzupełnij ścieżkę do checkpointa:
convnext_model.load_state_dict(torch.load(""))

### 3.5. ConvNeXt V2

In [ ]:
convnextv2_model = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-base-1k-224")

# Add on classifier
n_inputs = convnextv2_model.classifier.in_features
convnextv2_model.classifier = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.GELU(), nn.Dropout(0.4),
    nn.Linear(256, N_CLASSES), nn.Softmax(dim=1))

# !!! TODO: Uzupełnij ścieżkę do checkpointa:
convnextv2_model.load_state_dict(torch.load(""))

## 4. Klasyfikacja zdjęć ulic i mieszkań

In [ ]:
# TODO: Znajdź 2 zdjęcia (jedno z Google Maps i jedno z Airbnb) przedstawiające budynek/pokój 
# należący do którejś z klas ze słownika LABELS.
# Następnie dokonaj klasyfikacji tego obrazu przy użyciu każdej z sieci -> Porównaj wyniki
# Wskazówka: Najlepiej użyć pochodzących z obszaru USA z racji, że modele trenowane były właśnie na 
# zdjęciach pochodzących z tego kraju.

image1 = Image.open("PATH-TO-IMAGE-FROM-GOOGLE-MAPS")
image2 = Image.open("PATH-TO-IMAGE-FROM-AIRBNB")

In [ ]:
image1 = img_transform(image1)
image2 = img_transform(image2)

In [ ]:
image1 = image1.to(device)
image2 = image2.to(device)

### 4.1. VGG-16

In [ ]:
vgg16_model = vgg16_model.to(device)
vgg16_model.eval()

vgg16_img1_out = vgg16_model(image1)
_, img1_class_vgg16 = torch.max(vgg16_img1_out[0], 1)

vgg16_img2_out = vgg16_model(image2)
_, img2_class_vgg16 = torch.max(vgg16_img2_out[0], 1)

### 4.2. ResNet-50

In [ ]:
resnet50_model = resnet50_model.to(device)
resnet50_model.eval()

resnet50_img1_out = vgg16_model(image1)
_, img1_class_resnet50 = torch.max(resnet50_img1_out[0], 1)

resnet50_img2_out = vgg16_model(image2)
_, img2_class_resnet50 = torch.max(resnet50_img2_out[0], 1)

### 4.3. Swin Transformer

In [ ]:
swin_model = swin_model.to(device)
swin_model.eval()

swin_img1_out = vgg16_model(image1)
_, img1_class_swin = torch.max(swin_img1_out[0], 1)

swin_img2_out = vgg16_model(image2)
_, img2_class_swin = torch.max(swin_img2_out[0], 1)

### 4.4. ConvNeXt

In [ ]:
convnext_model = convnext_model.to(device)
convnext_model.eval()

convnext_img1_out = convnext_model(image1)
_, img1_class_convnext = torch.max(convnext_img1_out[0], 1)

convnext_img2_out = convnext_model(image2)
_, img2_class_convnext = torch.max(convnext_img2_out[0], 1)

### 4.5. ConvNeXt V2

In [ ]:
convnextv2_model = convnextv2_model.to(device)
convnextv2_model.eval()

convnextv2_img1_out = convnextv2_model(image1)
_, img1_class_convnextv2 = torch.max(convnextv2_img1_out[0], 1)

convnextv2_img2_out = convnextv2_model(image2)
_, img2_class_convnextv2 = torch.max(convnextv2_img2_out[0], 1)

## 5. Interpretowalność predykcji

In [11]:
# TODO: Dokonaj analizy interpretowalności dla poszczególnych modeli i wybranych wcześniej obrazów.

### 5.1. VGG-16

In [ ]:
# Objekt IntegratedGradient i atrybuty
integrated_gradients_vgg16 = IntegratedGradients(vgg16_model)

attributions_ig_vgg16_img1 = integrated_gradients_vgg16.attribute(image1, target=img1_class_vgg16, n_steps=30)
attributions_ig_vgg16_img2 = integrated_gradients_vgg16.attribute(image2, target=img2_class_vgg16, n_steps=30)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_vgg16_img1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_vgg16_img2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

### 5.2. ResNet-50

In [ ]:
# Objekt IntegratedGradient i atrybuty
integrated_gradients_resnet = IntegratedGradients(resnet50_model)

attributions_ig_resnet_img1 = integrated_gradients_resnet.attribute(image1, target=img1_class_resnet50, n_steps=30)
attributions_ig_resnet_img2 = integrated_gradients_resnet.attribute(image2, target=img2_class_resnet50, n_steps=30)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_resnet_img1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_resnet_img2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

### 5.3. Swin Transformer

In [ ]:
# Objekt IntegratedGradient i atrybuty
integrated_gradients_swin = IntegratedGradients(swin_model)

attributions_ig_swin_img1 = integrated_gradients_swin.attribute(image1, target=img1_class_swin, n_steps=30)
attributions_ig_swin_img2 = integrated_gradients_swin.attribute(image2, target=img2_class_swin, n_steps=30)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_swin_img1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_swin_img2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

### 5.4. ConvNeXt

In [ ]:
# Objekt IntegratedGradient i atrybuty
integrated_gradients_convnext = IntegratedGradients(convnext_model)

attributions_ig_convnext_img1 = integrated_gradients_convnext.attribute(image1, target=img1_class_convnext, n_steps=30)
attributions_ig_convnext_img2 = integrated_gradients_convnext.attribute(image2, target=img2_class_convnext, n_steps=30)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_convnext_img1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_convnext_img2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

### 5.5. ConvNeXt V2

In [ ]:
# Objekt IntegratedGradient i atrybuty
integrated_gradients_convnextv2 = IntegratedGradients(convnextv2_model)

attributions_ig_convnextv2_img1 = integrated_gradients_convnextv2.attribute(image1, target=img1_class_convnextv2, n_steps=30)
attributions_ig_convnextv2_img2 = integrated_gradients_convnextv2.attribute(image2, target=img2_class_convnextv2, n_steps=30)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_convnextv2_img1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image1.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)

In [ ]:
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_convnextv2_img2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             np.transpose(image2.squeeze(dim=0).cpu().detach().numpy(), (1,2,0)),
                             methods=["original_image", "heat_map"],
                             signs=['all', 'positive'],
                             cmap="viridis",
                             show_colorbar=True)